# Clothes

### Data Format: (examples, features)

* Input (60000, 28, 28, 1)
* Output (60000,)
* X_train  -- (60000,28,28, 1)
* X_test  -- (10000,28,28, 1)
* Y_train  -- (60000, 10)
* Y_test  -- (10000, 10)



### Model - 5 Layer Conv Network

    1. Conv --> Conv --> Maxpool --> Droupout ( 28,28,1(Input) > 26,26,32 > 24,24,32 > 12,12,32 > 12,12,32 )
    2. Conv --> Conv --> Maxpool --> Droupout ( 12,12,64 > 12,12,64 > 6,6,64 > 6,6,64 )
    3. Conv --> Dropout --> Flatten  (6,6,128 > 6,6,128 > 4608)
    4. Dense --> BN --> Dropout   (128 > 128 > 128)
    5. Dense --> Softmax   (10 (Output))


### Results
* Accuracy: 91% (can increase with more epochs)


In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.layers.normalization import BatchNormalization

C:\Users\abido\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Load the dataset

In [12]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()

In [13]:
print(X_train.shape)
print(X_test.shape)

(60000, 28, 28)
(10000, 28, 28)


### Preprocessing

In [14]:
#X_train = X_train / 255.0
#X_test = X_test / 255.0

X_train = X_train.reshape([-1, 28, 28, 1])
X_test = X_test.reshape([-1, 28, 28, 1])

In [15]:
def convert_to_one_hot(Y, C):
    """
    Y: (1, number of examples)
    C: number of classes
    returns: (number of classes, C)
    """
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

Y_train = convert_to_one_hot(Y_train, 10)
Y_test = convert_to_one_hot(Y_test, 10)

In [16]:
print("X_test shape: {}".format(X_train.shape))
print("Y_test shape: {}".format(Y_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("Y_test shape: {}".format(Y_test.shape))

X_test shape: (60000, 28, 28, 1)
Y_test shape: (60000, 10)
X_test shape: (10000, 28, 28, 1)
Y_test shape: (10000, 10)


### Model

In [17]:
# parameters
input_shape = (28,28,1)
num_classes = 10
num_epochs = 5
batch_size= 128

In [18]:
def createModel():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),data_format='channels_last' ,activation='relu',kernel_initializer='he_normal',input_shape=input_shape))
    model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',kernel_initializer='he_normal'))
    model.add(MaxPool2D((2, 2)))
    model.add(Dropout(0.20))
    
    model.add(Conv2D(64, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
    model.add(Conv2D(64, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

In [19]:
model = createModel()
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 12, 12, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 6, 6, 64)          0         
__________

In [20]:
history = model.fit(X_train, Y_train, epochs=num_epochs, batch_size= batch_size, verbose=1)

Epoch 1/5
60000/60000 [==============================] - 215s 4ms/step - loss: 0.5353 - acc: 0.8082
Epoch 2/5
60000/60000 [==============================] - 211s 4ms/step - loss: 0.3498 - acc: 0.8728
Epoch 3/5
60000/60000 [==============================] - 206s 3ms/step - loss: 0.2989 - acc: 0.8919
Epoch 4/5
60000/60000 [==============================] - 223s 4ms/step - loss: 0.2696 - acc: 0.9015
Epoch 5/5
60000/60000 [==============================] - 205s 3ms/step - loss: 0.2495 - acc: 0.9094


### Results

In [21]:
[loss, acc] = model.evaluate(X_test, Y_test, verbose = 1)

print("Test Accuracy is % {:.2f}%".format(acc*100))

preds = model.predict(X_test, verbose=1)
preds = np.argmax(preds, axis = 1)

10000/10000 [==============================] - 13s 1ms/step
Test Accuracy is % 90.55%
10000/10000 [==============================] - 12s 1ms/step


In [22]:
preds[:5]

array([9, 2, 1, 1, 6], dtype=int64)